# T1134 Access Token Manipulation

-----------------------------------------------------------------------

## Technique Description

Adversaries may modify access tokens to operate under a different user or system security context to perform actions and bypass access controls. Windows uses access tokens to determine the ownership of a running process. A user can manipulate access tokens to make a running process appear as though it is the child of a different process or belongs to someone other than the user that started the process. When this occurs, the process also takes on the security context associated with the new token.

An adversary can use built-in Windows API functions to copy access tokens from existing processes; this is known as token stealing. These token can then be applied to an existing process (i.e. [Token Impersonation/Theft](https://attack.mitre.org/techniques/T1134/001)) or used to spawn a new process (i.e. [Create Process with Token](https://attack.mitre.org/techniques/T1134/002)). An adversary must already be in a privileged user context (i.e. administrator) to steal a token. However, adversaries commonly use token stealing to elevate their security context from the administrator level to the SYSTEM level. An adversary can then use a token to authenticate to a remote system as the account for that token if the account has appropriate permissions on the remote system.(Citation: Pentestlab Token Manipulation)

Any standard user can use the <code>runas</code> command, and the Windows API functions, to create impersonation tokens; it does not require access to an administrator account. There are also other mechanisms, such as Active Directory fields, that can be used to modify access tokens.

## Technique Detection

If an adversary is using a standard command-line shell, analysts can detect token manipulation by auditing command-line activity. Specifically, analysts should look for use of the <code>runas</code> command. Detailed command-line logging is not enabled by default in Windows.(Citation: Microsoft Command-line Logging)

If an adversary is using a payload that calls the Windows token APIs directly, analysts can detect token manipulation only through careful analysis of user network activity, examination of running processes, and correlation with other endpoint and network behavior. 

There are many Windows API calls a payload can take advantage of to manipulate access tokens (e.g., <code>LogonUser</code> (Citation: Microsoft LogonUser), <code>DuplicateTokenEx</code>(Citation: Microsoft DuplicateTokenEx), and <code>ImpersonateLoggedOnUser</code>(Citation: Microsoft ImpersonateLoggedOnUser)). Please see the referenced Windows API pages for more information.

Query systems for process and thread token information and look for inconsistencies such as user owns processes impersonating the local SYSTEM account.(Citation: BlackHat Atkinson Winchester Token Manipulation)

Look for inconsistencies between the various fields that store PPID information, such as the EventHeader ProcessId from data collected via Event Tracing for Windows (ETW), Creator Process ID/Name from Windows event logs, and the ProcessID and ParentProcessID (which are also produced from ETW and other utilities such as Task Manager and Process Explorer). The ETW provided EventHeader ProcessId identifies the actual parent process.

-----------------------------------------------------------------------

### Tactics:

  * Defense-Evasion

  * Privilege-Escalation

### Platforms:

  * Windows

### Adversary Required Permissions:

  * User

  * Administrator

### Effective Permissions:

  * SYSTEM

### Defenses Bypassed:

  * Windows User Account Control

  * Heuristic Detection

  * System Access Controls

  * Host Forensic Analysis

### Data Sources:

  * **Process:** Process Metadata

  * **Process:** Process Creation

  * **Command:** Command Execution

  * **Active Directory:** Active Directory Object Modification

  * **Process:** OS API Execution

  * **User Account:** User Account Metadata

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| Blue Mockingbird | [Blue Mockingbird](https://attack.mitre.org/groups/G0108) has used JuicyPotato to abuse the <code>SeImpersonate</code> token privilege to escalate from web application pool accounts to NT Authority\SYSTEM.(Citation: RedCanary Mockingbird May 2020)| 
| FIN6 | [FIN6](https://attack.mitre.org/groups/G0037) has used has used Metasploit’s named-pipe impersonation technique to escalate privileges.(Citation: FireEye FIN6 Apr 2019)| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1134)

  * [Blackhat Atkinson Winchester Token Manipulation](https://www.blackhat.com/docs/eu-17/materials/eu-17-Atkinson-A-Process-Is-No-One-Hunting-For-Token-Manipulation.pdf), Atkinson, J., Winchester, R. (2017, December 7). A Process is No One: Hunting for Token Manipulation. Retrieved December 21, 2017.

  * [Microsoft Command-Line Logging](https://technet.microsoft.com/en-us/windows-server-docs/identity/ad-ds/manage/component-updates/command-line-process-auditing), Mathers, B. (2017, March 7). Command line process auditing. Retrieved April 21, 2017.

  * [Microsoft Logonuser](https://msdn.microsoft.com/en-us/library/windows/desktop/aa378184(v=vs.85).aspx), Microsoft TechNet. (n.d.). Retrieved April 25, 2017.

  * [Microsoft Duplicatetokenex](https://msdn.microsoft.com/en-us/library/windows/desktop/aa446617(v=vs.85).aspx), Microsoft TechNet. (n.d.). Retrieved April 25, 2017.

  * [Microsoft Impersonateloggedonuser](https://msdn.microsoft.com/en-us/library/windows/desktop/aa378612(v=vs.85).aspx), Microsoft TechNet. (n.d.). Retrieved April 25, 2017.

  * [Pentestlab Token Manipulation](https://pentestlab.blog/2017/04/03/token-manipulation/), netbiosX. (2017, April 3). Token Manipulation. Retrieved April 21, 2017.

  * [Capec](https://capec.mitre.org/data/definitions/633.html)

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 20 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres, Aaron Diaz

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- N/A

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Example APT/Threat actor | 1, 2 | 2, 3, 5|

## Detection Blindspots

- Detecting this activity on the network requires visibility into TLS connections and effective network signatures.
- Traffic may not be able to be seen with the normal toolset, may need to get access to existing tools outside of normal processes and deploy new technology.
- Requires sufficient GPO auditing to be enabled to ingest the access logs needed to identify behaviors of token theft.

## Analytical References

  * https://github.com/nettitude/PoshC2/blob/master/resources/modules/Invoke-TokenManipulation.ps1
  * https://github.com/cobbr/SharpSploit/blob/master/SharpSploit/SharpSploit%20-%20Quick%20Command%20Reference.md#sharpsploitcredentialstokens
  * https://medium.com/palantir/windows-privilege-abuse-auditing-detection-and-defense-3078a403d74e
  * https://posts.specterops.io/understanding-and-defending-against-access-token-theft-finding-alternatives-to-winlogon-exe-80696c8a73b
  * https://github.com/hatRiot/token-priv/blob/master/abusing_token_eop_1.0.txt
  * https://www.ired.team/offensive-security/privilege-escalation/t1134-access-token-manipulation
  * https://www.blackhat.com/docs/eu-17/materials/eu-17-Atkinson-A-Process-Is-No-One-Hunting-For-Token-Manipulation.pdf
  * https://niiconsulting.com/checkmate/2019/11/token-manipulation-attacks-part-1-introduction-to-tokens-and-privileges/
  * https://blog.netspi.com/tokenvator-a-tool-to-elevate-privilege-using-windows-tokens/
  * https://docs.microsoft.com/en-us/windows/security/threat-protection/security-policy-settings/create-a-token-object
  * https://docs.microsoft.com/en-us/windows/security/threat-protection/security-policy-settings/replace-a-process-level-token
  * https://docs.microsoft.com/en-us/windows/security/threat-protection/auditing/event-4672

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Turla uses the sub technique of this. Please refer to [create a process with a token](http://10.24.35.157/lab/tree/playbooks/Enterprise/Privilege%20Escalation/T1134.002%20Create%20Process%20With%20Token.ipynb)

#### Analytic 1

  * **Information:** N/A

  * **Source:** GPO Auditing:
    - Audit Special Logon
    - Audit Sensitive Privilege Use
    - Audit Non Sensitive Privilege Use
    - Authorization Policy Change

  * **Tool:** Kibana

  * **Notes:** event_id: 4672 followed by a event_id: 4674 within a short time period can indicate possible token theft. Will likely see a Logon Type 2 (interactive)
    or Logon Type 3 (network).

  * **Query_1:** ```event_id: 4672```
      - Special privileges assigned to new logon
  
  * **Query_2:** ```event_id: 4674```
      - An operation was attempted on a privileged object


#### Analytic 2

  * **Information:** N/A

  * **Source:** GPO Auditing:
    - Audit Special Logon
    - Audit Sensitive Privilege Use
    - Audit Non Sensitive Privilege Use
    - Authorization Policy Change
    
  * **Tool:** Kibana

  * **Notes:** Monitoring for abused Privileges being enabled like SeDebugPrivilege is a good indicator to investigate further.

  * **Query:** ```event_id: 4703```
      - user right was adjusted


#### Analytic 3

  * **Information:** N/A

  * **Source:** GPO Auditing:
    - Audit Special Logon
    - Audit Sensitive Privilege Use
    - Audit Non Sensitive Privilege Use
    - Authorization Policy Change
    
  * **Tool:** Kibana

  * **Notes:** event_id: 4763 followed by event_id: 4674 within a short time period can indicate possible malicious token creation. Monitoring for processes that request/invoke a privilege like SeTcpPrivilege (4763) Once granted, the calling process has the ability to generate a new token with additional privileges for impersonation.

  * **Query_1:** ```event_id: 4673```
      - A privileged service was called
  
  * **Query_2:** ```event_id: 4674```
      - An operation was attempted on a privileged object


#### Analytic 4

  * **Information:** N/A

  * **Source:** SACL auditing for process objects using NtObjectManager or something similar
    
  * **Tool:** Kibana

  * **Notes:** Attacker attempting to "upgrade" from an Administrator context to SYSTEM via Token Stealing. Access rights needed to obtain the access token from a process object are:
    - PROCESS_QUERY_INFORMATION
    - PROCESS_QUERY_LIMITED_INFORMATION 
    When SACL is auditing, any process using these rights to access a process object will fire an event for success/failure access to the object.

  * **Query_1:** ```event_id: 4656  with the ACCESS_MASK == 0x1400 (PROCESS_QUERY_INFORMATION)```
  
  * **Query_2:** ```ACCESS_MASK == 0x1000 (PROCESS_QUERY_LIMITED_INFORMATION) followed by event_id: 4663 ```



-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------